In [ ]:
import numpy as np
from tensorflow.keras.models import load_model
from PIL import Image
import matplotlib.pyplot as plt

In [ ]:
pip install tensorflow==2.17.0

In [ ]:
import tensorflow as tf
print(tf.__version__)

2.17.0


In [ ]:
# Memuat model jerawat dan model jenis kulit
acne_model = load_model("/content/acne_detection (4).h5")
skin_type_model = load_model("/content/skintypes_detection_model.h5")

FileNotFoundError: [Errno 2] Unable to synchronously open file (unable to open file: name = '/content/acne_detection (4).h5', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)

In [ ]:
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt

# Fungsi preprocess untuk mempersiapkan gambar (dengan ukuran target yang sama)
def preprocess_image(image_path, target_size=(128, 128)):
    image = Image.open(image_path)
    image = image.resize(target_size)
    image_array = np.array(image) / 255.0  # Normalisasi gambar
    image_array = np.expand_dims(image_array, axis=0)  # Menambahkan batch dimension
    return image_array

# Fungsi untuk mendapatkan nama label berdasarkan hasil prediksi (Acne)
def get_acne_label_name(prediction):
    # Asumsi prediction adalah output dari model, berupa probabilitas untuk kelas 'Yes' (Acne)
    predicted_class = "Acne" if prediction[0][0] > 0.5 else "Non-Acne"
    return predicted_class

# Fungsi untuk mendapatkan nama label berdasarkan hasil one-hot encoding (Jenis Kulit)
def get_skin_type_label_name(label):
    skin_type_labels = {0: 'dry', 1: 'normal', 2: 'oily'}
    return skin_type_labels.get(label, "Unknown")

def predict_acne_and_skin_type(image_path, acne_model, skin_type_model):
    # Preprocess gambar sekali dengan ukuran target 128x128
    image = preprocess_image(image_path, target_size=(128, 128))

    # Prediksi dengan model acne
    results_acne = acne_model.predict(image)
    acne_label_name = get_acne_label_name(results_acne)  # Gunakan fungsi yang baru diubah

    # Menambahkan logika untuk meningkatkan keakuratan
    print(f"Prediksi Acne (model): {acne_label_name}")

    # Prediksi dengan model skin type
    results_skin_type = skin_type_model.predict(image)
    skin_type_label = np.argmax(results_skin_type, axis=1)[0]  # Ambil hasil prediksi jenis kulit
    skin_type_label_name = get_skin_type_label_name(skin_type_label)

    # Kombinasi hasil
    combined_results = {
        "acne_detection": acne_label_name,
        "skin_type_detection": skin_type_label_name
    }

    return combined_results


# Fungsi untuk menampilkan gambar
def show_image(image_path):
    image = Image.open(image_path)
    plt.imshow(image)
    plt.axis("off")
    plt.show()

# # Input path gambar

# image_path = '/content/60d1954a-9631-4c9a-9295-febe81ab-20221107104411.jpg'
# image_path = '/content/0_19hJvhBaJyf8eJwT_.jpg'

# # Menampilkan gambar
# show_image(image_path)

# # Melakukan prediksi gabungan dengan model acne dan skin type
# # Pastikan acne_model dan skin_type_model sudah didefinisikan sebelumnya
# combined_results = predict_acne_and_skin_type(image_path, acne_model, skin_type_model)
image_path = '/content/download.jpeg'
preprocessed_image = preprocess_image(image_path, target_size=(128, 128))

# Prediksi dengan model acne
acne_prediction = acne_model.predict(preprocessed_image)
print("Prediksi Acne:", get_acne_label_name(acne_prediction))  # Gunakan fungsi baru

# Prediksi dengan model skin type
skin_type_prediction = skin_type_model.predict(preprocessed_image)
skin_type_label = np.argmax(skin_type_prediction, axis=1)[0]
print("Prediksi Jenis Kulit:", get_skin_type_label_name(skin_type_label))

# Menampilkan gambar
show_image(image_path)


In [ ]:
# import pandas as pd
# # acne_model = load_model("/content/drive/MyDrive/BEAUSKIN ML/model/acne_detection.h5")
# # skin_type_model = load_model("/content/drive/MyDrive/BEAUSKIN ML/model/skintypes_detection_model.h5")
# # Memuat dataset skincare
# skincare = pd.read_csv('/content/drive/MyDrive/BEAUSKIN ML/skincare dataset/export_skincare.csv')

# # Fungsi untuk memberikan rekomendasi produk berdasarkan jenis kulit dan kondisi jerawat
# def recommend_products(skin_type, acne_condition, skincare_data):
#     # Filter produk berdasarkan jenis kulit
#     recommended_products = skincare_data[
#         (skincare['skintype'] == skintype)
#       # (skincare['condition'] == acne_condition)
#     ]

#     if recommended_products.empty:
#         return "Tidak ada produk yang cocok dengan kondisi kulit Anda."
#     else:
#         return recommended_products[['product_name', 'brand', 'description']]

# # Mengambil rekomendasi berdasarkan hasil deteksi
# skintype = skin_type_label_name  # misalnya: 'oily'
# acne_condition = "Acne" if acne_label_name == "Acne" else "No Acne"

# # Menampilkan rekomendasi produk
# recommended_products = recommend_products(skintype, acne_condition, skincare)

# print("Rekomendasi Produk:")
# print(recommended_products)


In [ ]:
import pandas as pd

# Memuat dataset skincare
skincare = pd.read_csv('/content/drive/MyDrive/BEAUSKIN ML/skincare dataset/export_skincare.csv')

# Fungsi untuk memberikan rekomendasi produk berdasarkan jenis kulit dan kondisi jerawat
def recommend_products(skin_type, acne_condition, skincare_data):
    # Filter produk berdasarkan jenis kulit dan kondisi jerawat
    recommended_products = skincare_data[
        (skincare_data['skintype'].str.contains(skin_type, case=False, na=False)) &  # Pastikan kolom cocok
        (skincare_data['notable_effects'].str.contains(acne_condition, case=False, na=False))
    ]

    if recommended_products.empty:
        return "Tidak ada produk yang cocok dengan kondisi kulit Anda."
    else:
        return recommended_products[['product_name', 'brand', 'description']].head(5)

# Mengambil rekomendasi berdasarkan hasil deteksi
skintype = skin_type_label_name  # Hasil deteksi jenis kulit, misalnya: 'oily'
acne_condition = "Acne" if acne_label_name == "Acne" else "No Acne"

# Menampilkan rekomendasi produk
recommended_products = recommend_products(skintype, acne_condition, skincare)

print("Rekomendasi Produk:")
print(recommended_products)
